In [5]:
import numpy as np

# Proximal operators

<img src="prox_2.png" width="60%">

In [6]:
def prox_od_1norm(A, l):
    """
    calculates the prox of the off-diagonal 1norm at a point A
    """
    
    (d1,d2) = A.shape
    res = np.sign(A) * np.maximum(np.abs(A) - l, 0)
    
    for i in np.arange(np.minimum(d1,d2)):
        res[i,i] = A[i,i]

    return res

# The log-determent function $\log\det|\Theta|$

Let $A$ be a square $n × n$ matrix with $n$ linearly independent eigenvectors $q_i$ (where $i = 1, ..., n$). 
Then $A$ can be factorized as

$\displaystyle \mathbf {A} =\mathbf {Q} \mathbf {D} \mathbf {Q}^{-1}$ 

where $Q$ is the square $n × n$ matrix whose $i$th column is the eigenvector $q_i$ of $A$, and $D$ is the diagonal matrix whose diagonal elements are the corresponding eigenvalues, $D_{ii} = λ_i$. 

We define the following functions in order to describe the proximal operator and  its derivative of log-determinant function 

$\log(\det|\Theta|) \leq \beta\log(\Theta)$,

$\beta > 0$:


<img src="log_det.png" width="60%">

In [7]:
def phip(d, beta):
    return 0.5 * (np.sqrt(d**2 + 4*beta) + d)
    B = Q @ np.diag(phip(D,beta)) @ Q.T
    return B

In [8]:
def phiplus(A, beta, D = np.array([]), Q = np.array([])):
    # D and Q are optional if already precomputed
    if len(D) != A.shape[0]:
        D, Q = np.linalg.eigh(A)
        print("Single eigendecomposition is executed in phiplus")
    
    B = Q @ np.diag(phip(D,beta)) @ Q.T #Q is orthogonal matrix, so inversed(Q) = transposed(Q)
    return B

# Covariance matrix

In [36]:
mean = [0, 0]
cov = [[1, 0], [0, 100]]  # diagonal covariance

In [37]:
#sample random covariance matrix
S = np.random.multivariate_normal(mean, cov, (2, 2))
S

array([[[-0.31243859,  9.66666487],
        [-0.91614224, -0.10878634]],

       [[-0.80603943, -9.5440804 ],
        [-1.05168752,  1.26878569]]])

# ADMM

In [38]:
max_iter = 10
p = 2
rho = 1
lambda1 = 0.01

Omega_t = np.eye(p)
Theta_t = np.eye(p)
X_t = np.zeros((p, p))
S = S[0,:,:]

for iter_t in np.arange(max_iter):
        # Omega Update
        W_t = Theta_t - X_t - (1 / rho) * S
        eigD, eigQ = np.linalg.eigh(W_t)
        Omega_t = phiplus(W_t, beta=1 / rho, D=eigD, Q=eigQ)

        # Theta Update
        Theta_t = prox_od_1norm(Omega_t + X_t, (1 / rho) * lambda1)

        # X Update
        X_t = X_t + Omega_t - Theta_t
        print("Omega Update \n", Omega_t)
        print("Theta Update \n", Theta_t)
        print("X Update \n", X_t, "\n")

Omega Update 
 [[1.91722461 0.682389  ]
 [0.682389   1.76553413]]
Theta Update 
 [[1.91722461 0.672389  ]
 [0.672389   1.76553413]]
X Update 
 [[0.   0.01]
 [0.01 0.  ]] 

Omega Update 
 [[2.72524584 1.31114268]
 [1.31114268 2.43009488]]
Theta Update 
 [[2.72524584 1.31114268]
 [1.31114268 2.43009488]]
X Update 
 [[0.   0.01]
 [0.01 0.  ]] 

Omega Update 
 [[3.48167728 1.93523079]
 [1.93523079 3.04632532]]
Theta Update 
 [[3.48167728 1.93523079]
 [1.93523079 3.04632532]]
X Update 
 [[0.   0.01]
 [0.01 0.  ]] 

Omega Update 
 [[4.2078227  2.55097197]
 [2.55097197 3.63412758]]
Theta Update 
 [[4.2078227  2.55097197]
 [2.55097197 3.63412758]]
X Update 
 [[0.   0.01]
 [0.01 0.  ]] 

Omega Update 
 [[4.91432881 3.16080438]
 [3.16080438 4.20360801]]
Theta Update 
 [[4.91432881 3.16080438]
 [3.16080438 4.20360801]]
X Update 
 [[0.   0.01]
 [0.01 0.  ]] 

Omega Update 
 [[5.60726534 3.76594576]
 [3.76594576 4.76056643]]
Theta Update 
 [[5.60726534 3.76594576]
 [3.76594576 4.76056643]]
X Update

In [39]:
Omega_t - Theta_t # the constraint is satisfied

array([[0., 0.],
       [0., 0.]])